## 한글텍스트 - 분류 모델링
### 로지스틱 회귀 모델 with TF-IDF

### 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# 경고메세지 무시
import warnings
warnings.filterwarnings(action='ignore') 

### 데이타 불러오기 - 텍스트 형태

In [3]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'nsmc_train_clean.csv'
TEST_CLEAN_DATA = 'nsmc_test_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [4]:
train_df = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA )
test_df = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA )

In [5]:
# 결측치 제거
train_df = train_df.dropna()
test_df = test_df.dropna()

In [16]:
test_df.shape

(49564, 3)

### TF-IDF 벡터화
- min_df : 설정한 값보다 특정 토큰의 df 값이 적게 나오면 벡터화 과정에서 제거한다는 의미
- max_df : 설정한 값보다 특정 토큰의 df 값이 크게 나오면 벡터화 과정에서 제거한다는 의미
- ngram_range : 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지를 지정하는 인자

#### 형태소 분석기 함수화

In [6]:
from konlpy.tag import Okt

okt = Okt()
def okt_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [7]:
okt_tokenizer("꼭 대박나세요")

['꼭', '대박나세요']

#### TF-IDF 피처벡터화

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Okt 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

### 로지스틱 회귀 모델 분류

In [11]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

lg_clf.fit(tfidf_matrix_train , train_df['label'] )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 테스트 데이타를 이용한 정확도 평가

In [12]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# 테스트 데이타 예측
preds = lg_clf.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'], preds))

Logistic Regression 정확도:  0.8625010087967073


In [13]:
# 예측값
pred = lg_clf.predict(tfidf_matrix_test)

In [14]:
# 실제값
test_df['label']

0        1
2        0
3        0
4        0
5        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: label, Length: 49564, dtype: int64

In [15]:
# 정확도
sum(pred == test_df['label'] )

42749

In [17]:
42749/49564

0.8625010087967073

### 로지스틱 회귀 모델 분류 & GridSearchCV

In [52]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   48.9s finished


{'C': 4.5} 0.8612


### 테스트 데이타에 TF-IDF 벡터 변환 후 정확도 평가

In [53]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.8650835283673635
